In [1]:
try:
    import dolfinx
except ImportError:
    !wget "https://github.com/fem-on-colab/fem-on-colab.github.io/raw/a312183/releases/fenicsx-install-real.sh" -O "/tmp/fenicsx-install.sh" && bash "/tmp/fenicsx-install.sh"
    import dolfinx

--2024-05-17 15:25:00--  https://github.com/fem-on-colab/fem-on-colab.github.io/raw/a312183/releases/fenicsx-install-real.sh
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/fem-on-colab/fem-on-colab.github.io/a312183439cf07dbe5561fe31ebee6ccd1f77324/releases/fenicsx-install-real.sh [following]
--2024-05-17 15:25:00--  https://raw.githubusercontent.com/fem-on-colab/fem-on-colab.github.io/a312183439cf07dbe5561fe31ebee6ccd1f77324/releases/fenicsx-install-real.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4319 (4.2K) [text/plain]
Saving to: ‘/tmp/fenicsx-install.sh’

/tmp/fenicsx-instal 100%[==

In [17]:
try:
    import viskex
except ImportError:
    !pip3 install "viskex@git+https://github.com/viskex/viskex.git@64c23fe"
    import viskex

  Cloning https://github.com/viskex/viskex.git (to revision 64c23fe) to /tmp/pip-install-d7a0mzva/viskex_77f730fb478c4b96810636fa1af5225e
  Running command git clone --filter=blob:none --quiet https://github.com/viskex/viskex.git /tmp/pip-install-d7a0mzva/viskex_77f730fb478c4b96810636fa1af5225e
  Running command git checkout -q 64c23fe
  Resolved https://github.com/viskex/viskex.git to commit 64c23fe
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 16.9 MB/s eta 0:00:00
  Created wheel for viskex: filename=viskex-0.0.dev1-py3-none-any.whl size=22891 sha256=01834622548d4ece28208f6d091a2b9f4b38459e57a40e7deb68dd6a36afc1ad
  Stored in directory: /tmp/pip-ephem-wheel-cache-io993p64/wheels/28/9e/ea/48f255983e2a985971e7b543c2cdfdd51df4273800b0fc726c
Successfully built viskex


In [4]:
try:
    import gmsh
except ImportError:
    !wget "https://github.com/fem-on-colab/fem-on-colab.github.io/raw/a312183/releases/gmsh-install.sh" -O "/tmp/gmsh-install.sh" && bash "/tmp/gmsh-install.sh"
    import gmsh

In [2]:
import basix.ufl
import dolfinx.fem
import dolfinx.fem.petsc
import dolfinx.io
import gmsh
import mpi4py.MPI
import numpy as np
import numpy.typing
import petsc4py.PETSc
import ufl
import viskex

In [8]:
mesh = dolfinx.mesh.create_unit_square(MPI.COMM_WORLD, 10, 10)

# Create connectivities required by the rest of the code
mesh.topology.create_connectivity(mesh.topology.dim - 1, mesh.topology.dim)

In [9]:
# velocity space element
V_element = basix.ufl.element("Lagrange", mesh.basix_cell(), 2, shape=(mesh.geometry.dim, ))
# tensor space element
Q_element = basix.ufl.element("Lagrange", mesh.basix_cell(), 1, shape=(mesh.geometry.dim,mesh.geometry.dim), symmetry=True)

In [10]:
# define the function space
V = dolfinx.fem.functionspace(mesh, V_element)
Q = dolfinx.fem.functionspace(mesh, Q_element)

In [11]:
# interpolate velocity in the nodal value
def u_eval(x: np.typing.NDArray[np.float64]) -> np.typing.NDArray[  # type: ignore[no-any-unimported]
        petsc4py.PETSc.ScalarType]:
    """Return the flat velocity profile at the inlet."""
    values = np.zeros((2, x.shape[1]))
    values[0, :] = np.sin(np.pi*x[0])
    values[1, :] = -np.pi*x[1]* np.cos(np.pi*x[1])
    return values

u = dolfinx.fem.Function(V,name="velocity")
u.interpolate(u_eval)
#https://fenicsproject.discourse.group/t/numerical-values-from-ufl-spatialcoordinate/11064/2

In [32]:
def tau11_eval(x: np.typing.NDArray[np.float64]) -> np.typing.NDArray[  # type: ignore[no-any-unimported]
        petsc4py.PETSc.ScalarType]:
    """Return the flat velocity profile at the inlet."""
    values = np.zeros((1, x.shape[1]))
    values[0, :] = np.sin(np.pi*x[0])
    return values
def tau12_eval(x: np.typing.NDArray[np.float64]) -> np.typing.NDArray[  # type: ignore[no-any-unimported]
        petsc4py.PETSc.ScalarType]:
    """Return the flat velocity profile at the inlet."""
    values = np.zeros((1, x.shape[1]))
    values[0, :] = - np.pi*x[1]*np.cos(np.pi*x[0])
    return values
def tau22_eval(x: np.typing.NDArray[np.float64]) -> np.typing.NDArray[  # type: ignore[no-any-unimported]
        petsc4py.PETSc.ScalarType]:
    """Return the flat velocity profile at the inlet."""
    values = np.zeros((1, x.shape[1]))
    values[0, :] = np.sin(np.pi*x[0])*np.cos(np.pi*x[1])
    return values

def tau_eval(x: np.typing.NDArray[np.float64]) -> np.typing.NDArray[  # type: ignore[no-any-unimported]
        petsc4py.PETSc.ScalarType]:
    values = np.zeros((4, x.shape[1]))
    tmp = np.sin(np.pi*x[0])
    values[0, :] = tau11_eval(x)
    values[1, :] = tau22_eval(x)
    values[2, :] = tau12_eval(x)

    return values



In [35]:
q_D = dolfinx.fem.Function(Q)
q_D.interpolate(tau_eval)
fdim = mesh.topology.dim - 1
boundary_facets = dolfinx.mesh.locate_entities_boundary(mesh, fdim, lambda x: np.full(x.shape[1], True, dtype=bool))
bc_D = dolfinx.fem.dirichletbc(q_D, dolfinx.fem.locate_dofs_topological(Q, fdim, boundary_facets))


In [ ]:
viskex.dolfinx.plot_vector_field(u, "u")

In [18]:
psi = ufl.TestFunction(Q)
tau = ufl.TrialFunction(Q)
dx=ufl.Measure("dx", domain=mesh, metadata={"quadrature_degree": 10})
Wi = 10.

In [37]:
F = ufl.inner(ufl.dot(u,ufl.grad(tau)), psi)*dx - ufl.inner(ufl.dot(ufl.grad(u).T, tau) + ufl.dot(ufl.grad(u), tau),psi)*dx
F += 1/Wi * ufl.inner(tau,psi)*dx

In [38]:
L = 1/Wi *ufl.inner(ufl.Identity(mesh.geometry.dim),psi)*dx

In [39]:
from dolfinx.fem.petsc import LinearProblem
problem = LinearProblem(F, L, bcs=[bc_D], petsc_options={"ksp_type": "preonly", "pc_type": "lu"})
uh = problem.solve()